# Multihead Attention


https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb

Lets test multiple attention heads, one for each input, with univariate and mutivariate data

Lets test wavelet threshold denoising

Due to a slip in the matrix profile function, the matrix profile was computed in reverse in the previous iteration, this is adressed and computed normally in this notebook

## Load data, functions

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import MultiHeadAttention, BatchNormalization, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pywt
import matrixprofile as mp
from keras import Model
from keras.layers import dot
import datetime 
from datetime import timedelta
import math

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [4]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [5]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [6]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [7]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [8]:
def generate_training_val_examples(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    #we are only lookign to forecast the min gas price
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))

    
    return X_train, y_train, X_val, y_val, scaler



In [10]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [11]:
def descale_y_retrun_metrics(yhat, y_val2):
    #We will use validation data that has not had outleirs limited, will be a different min/max scaler as such
    dict_indexes=[]
    dict_dfs=[]
    for j in range(0, n_steps_out):
        RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
        for i in range(0, len(inputs)):  
            pred_descaled= (scaler.inverse_transform(yhat[:,j:j+1,:].reshape(yhat.shape[0], yhat.shape[2])))[:, i:i+1]
            groud_truth_descaled= ((scaler2.inverse_transform(y_val2[:,j:j+1,:].reshape(y_val2.shape[0], y_val2.shape[2]))))[:, i:i+1]
            RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
            MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
            MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
            MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
            R2 = r2_score(groud_truth_descaled, pred_descaled)
            RMSE_list.append(RMSE)
            MAE_list.append(MAE)
            MAPE_list.append(MAPE)
            R2_list.append(R2)
            MSE_list.append(MSE)
        metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)
        dict_dfs.append(metrics_df)
        dict_indexes.append('Lookahead' +str(j))
    metrics_dict = dict(zip(dict_indexes, dict_dfs))
    return metrics_dict

In [12]:
def generate_training_val_examples_univariate_output(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM, filter output to only the first input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))[:,:,:1]
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))[:,:,:1]

    
    return X_train, y_train, X_val, y_val, scaler

In [13]:
def descale_y_retrun_metrics_univariate_y(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for single output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    yhat_stretched= np.repeat(yhat[:,:,0],len(inputs)).reshape(yhat.shape[0], yhat.shape[1], len(inputs))
    y_val2_stretched= np.repeat(y_val2[:,:,0],len(inputs)).reshape(y_val2.shape[0], y_val2.shape[1], len(inputs))
    
    for j in range(0, n_steps_out):
        
        
        pred_descaled= (scaler.inverse_transform(yhat_stretched[:, j:j+1, :].reshape(yhat_stretched.shape[0], yhat_stretched.shape[2])))[:,:1]
        groud_truth_descaled= (scaler.inverse_transform(array([y_val2[ :, j:j+1,0].reshape(y_val2.shape[0])]*len(inputs)).transpose()))[:,:1]
        RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
        MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
        MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
        MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
        R2 = r2_score(groud_truth_descaled, pred_descaled)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)
    metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1, (n_steps_out+1)))

 
    return metrics_df

In [14]:
def add_mp(data, window):
    #Given 3d array, add matrix profile of (x,y,0) as new dimension; new array has dimensiosn (x,y,z+1) 
    mp_list=[]
    for i in data[:,:,0]:
        profile = mp.compute(i, window, n_jobs=4)['mp']
        #we are padding the end of the sequence with the mean
        #matrix profile is always 1 full window size smalelr than input data
        mp_list.append(np.append(([mean(profile)]*(data.shape[1]-len(profile))),profile))
        
    #concatenate matrix profile data with original    
    mp_array = np.array(mp_list).reshape(data.shape[0], data.shape[1])
    std_array = ((mp_array-mean(mp_array))/np.std(mp_array)).reshape(data.shape[0], data.shape[1],1)
    data = np.concatenate((data, std_array), axis=2)[:, window:, :]
    
    return data

In [15]:
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [16]:
def wavelet_denoising(x, wavelet='db4', level=2, tmod=1):
    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/tmod) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(3 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

In [17]:
def denoise_array(data, variable, wavelet, level, tmod):
    denoised_examples=[]
    for i in (data[:,:,variable]):
        
        denoised_examples.append(wavelet_denoising(i, wavelet=wavelet, level=level, tmod=tmod))
        denoised_array= np.array(denoised_examples)
        denoised_array= denoised_array.reshape(denoised_array.shape[0], denoised_array.shape[1], 1)
    return np.concatenate((denoised_array, data[:,:,1:]),axis=2)

## 2 layer, all variables, MP

In [18]:
def LSTM_model(): 
    
    #set up callback for best val loss model
    checkpoint_filepath='./cnn/checkpoint'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    
    n_hidden = 30
    input_train = Input(shape=(n_steps_in, X_train.shape[2]),name='input')
    output_train = Input(shape=( y_train.shape[1], y_train.shape[2]))




    enc_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('encoder' +str(i)))(input_train)

        decoder_input = RepeatVector(y_train.shape[1], name='repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('attention_dot'+str(i)))
        attention = Activation('softmax', name='attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='Context'+str(i))
        enc_head_list.append(context)
    enc_concat_attention = Concatenate(axis=2)(enc_head_list)
    
    
    
    dec_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('decoder' +str(i)))(enc_concat_attention)

        decoder_input = RepeatVector(y_train.shape[1],name='dec_repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('dec_alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('dec_attention_dot'+str(i)))
        attention = Activation('softmax', name='dec_attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='dec_Context'+str(i))
        dec_head_list.append(context)
    dec_concat_attention = Concatenate(axis=2)(dec_head_list)





    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)




    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)

    model = Model(inputs=input_train, outputs=out)

    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae'])
    return model, model_checkpoint_callback, checkpoint_filepath

In [24]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0,1,2,3,4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    #X_train= denoise_array(X_train, 0, 'db4', 2, 3)
    #X_val= denoise_array(X_val, 0, 'db4', 2, 3)
    X_train = add_mp(X_train, mp_window)
    X_val = add_mp(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_forward_10_lookahead/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_forward_10_lookahead/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_forward_10_lookahead/val_loss.csv')
    np.save("2_Att_layer_MP_forward_10_lookahead/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_forward_10_lookahead/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 225s 1s/step - loss: 0.2804 - mae: 0.2694 - val_loss: 0.0563 - val_mae: 0.1682
Epoch 2/15
  2/101 [..............................] - ETA: 1:39 - loss: 0.1297 - mae: 0.2364

KeyboardInterrupt: 

Loop was stopped during the final month of training; run for final month only

In [32]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    #X_train= denoise_array(X_train, 0, 'db4', 2, 3)
    #X_val= denoise_array(X_val, 0, 'db4', 2, 3)
    X_train = add_mp(X_train, mp_window)
    X_val = add_mp(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_forward_10_lookahead/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_forward_10_lookaheadm4/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_forward_10_lookaheadm4/val_loss.csv')
    np.save("2_Att_layer_MP_forward_10_lookaheadm4/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_forward_10_lookaheadm4/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 237s 1s/step - loss: 0.1244 - mae: 0.1952 - val_loss: 0.0493 - val_mae: 0.1550
Epoch 2/15
101/101 [==============================] - 127s 1s/step - loss: 0.0780 - mae: 0.1521 - val_loss: 0.0457 - val_mae: 0.1560
Epoch 3/15
101/101 [==============================] - 127s 1s/step - loss: 0.0641 - mae: 0.1420 - val_loss: 0.0397 - val_mae: 0.1283
Epoch 4/15
101/101 [==============================] - 129s 1s/step - loss: 0.0664 - mae: 0.1384 - val_loss: 0.0547 - val_mae: 0.1836
Epoch 5/15
101/101 [==============================] - 128s 1s/step - loss: 0.0557 - mae: 0.1340 - val_loss: 0.0474 - val_mae: 0.1600
Epoch 6/15
101/101 [==============================] - 128s 1s/step - loss: 0.0508 - mae: 0.1298 - val_loss: 0.0507 - val_mae: 0.1733
Epoch 7/15
101/101 [==============================] - 129s 1s/step - loss: 0.0489 - mae: 0.1279 - val_loss: 0.0521 - val_mae: 0.1786
Epoch 8/15
101/101 [==============================] - 127s 1s/step - 

Append month 4 to rest of results

In [25]:
train_m4 = np.load('2_Att_layer_MP_forward_10_lookaheadm4/training_metrics.npy',allow_pickle=True)
val_m4 = np.load("2_Att_layer_MP_forward_10_lookaheadm4/val_metrics.npy",allow_pickle=True)

In [26]:
train_m0_3 = np.load('2_Att_layer_MP_forward_10_lookahead/training_metrics.npy',allow_pickle=True)
val_m0_3 = np.load("2_Att_layer_MP_forward_10_lookahead/val_metrics.npy",allow_pickle=True)

In [28]:
train_metrics=np.concatenate((train_m0_3, train_m4), axis=0)
val_metrics = np.concatenate((val_m0_3, val_m4), axis=0)        

In [32]:
np.save("2_Att_layer_MP_forward_10_lookahead/training_metrics_all.npy", train_metrics)
np.save("2_Att_layer_MP_forward_10_lookahead/val_metrics_all.npy", val_metrics)

## DB4 wavelet

In [23]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'db4', 2, 3)
    X_val= denoise_array(X_val, 0, 'db4', 2, 3)
    X_train = add_mp(X_train, mp_window)
    X_val = add_mp(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_forward_10_lookahead_db4/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_forward_10_lookahead_db4/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_forward_10_lookahead_db4/val_loss.csv')
    np.save("2_Att_layer_MP_forward_10_lookahead_db4/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_forward_10_lookahead_db4/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 228s 1s/step - loss: 0.2912 - mae: 0.2796 - val_loss: 0.0590 - val_mae: 0.1783
Epoch 2/15
101/101 [==============================] - 137s 1s/step - loss: 0.2296 - mae: 0.2217 - val_loss: 0.0574 - val_mae: 0.1551
Epoch 3/15
101/101 [==============================] - 147s 1s/step - loss: 0.2204 - mae: 0.2119 - val_loss: 0.0562 - val_mae: 0.1501
Epoch 4/15
101/101 [==============================] - 148s 1s/step - loss: 0.2155 - mae: 0.2095 - val_loss: 0.0913 - val_mae: 0.2228
Epoch 5/15
101/101 [==============================] - 150s 1s/step - loss: 0.2086 - mae: 0.2067 - val_loss: 0.0543 - val_mae: 0.1581
Epoch 6/15
101/101 [==============================] - 149s 1s/step - loss: 0.2005 - mae: 0.2022 - val_loss: 0.0632 - val_mae: 0.1705
Epoch 7/15
101/101 [==============================] - 148s 1s/step - loss: 0.1986 - mae: 0.2012 - val_loss: 0.0623 - val_mae: 0.1678
Epoch 8/15
101/101 [==============================] - 148s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 237s 1s/step - loss: 0.6606 - mae: 0.4546 - val_loss: 0.4578 - val_mae: 0.4465
Epoch 2/15
107/107 [==============================] - 135s 1s/step - loss: 0.5496 - mae: 0.3874 - val_loss: 0.3988 - val_mae: 0.3655
Epoch 3/15
107/107 [==============================] - 136s 1s/step - loss: 0.5374 - mae: 0.3793 - val_loss: 0.4337 - val_mae: 0.4236
Epoch 4/15
107/107 [==============================] - 135s 1s/step - loss: 0.5095 - mae: 0.3716 - val_loss: 0.4339 - val_mae: 0.4038
Epoch 5/15
107/107 [==============================] - 136s 1s/step - loss: 0.4903 - mae: 0.3648 - val_loss: 0.4502 - val_mae: 0.4031
Epoch 6/15
107/107 [==============================] - 135s 1s/step - loss: 0.4632 - mae: 0.3559 - val_loss: 0.4793 - val_mae: 0.4754
Epoch 7/15
107/107 [==============================] - 135s 1s/step - loss: 0.4567 - mae: 0.3578 - val_loss: 0.5272 - val_mae: 0.4873
Epoch 8/15
107/107 [==============================] - 135s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 221s 1s/step - loss: 0.1699 - mae: 0.2372 - val_loss: 0.3467 - val_mae: 0.2527
Epoch 2/15
101/101 [==============================] - 126s 1s/step - loss: 0.1486 - mae: 0.2100 - val_loss: 0.3606 - val_mae: 0.2573
Epoch 3/15
101/101 [==============================] - 126s 1s/step - loss: 0.1426 - mae: 0.2078 - val_loss: 0.3822 - val_mae: 0.2634
Epoch 4/15
101/101 [==============================] - 127s 1s/step - loss: 0.1335 - mae: 0.1990 - val_loss: 0.3509 - val_mae: 0.2809
Epoch 5/15
101/101 [==============================] - 127s 1s/step - loss: 0.1283 - mae: 0.1965 - val_loss: 0.3537 - val_mae: 0.3027
Epoch 6/15
101/101 [==============================] - 127s 1s/step - loss: 0.1229 - mae: 0.1920 - val_loss: 0.3440 - val_mae: 0.2707
Epoch 7/15
101/101 [==============================] - 126s 1s/step - loss: 0.1173 - mae: 0.1868 - val_loss: 0.3690 - val_mae: 0.2952
Epoch 8/15
101/101 [==============================] - 127s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
82/82 [==============================] - 199s 1s/step - loss: 0.2155 - mae: 0.2137 - val_loss: 0.0587 - val_mae: 0.1524
Epoch 2/15
82/82 [==============================] - 109s 1s/step - loss: 0.1576 - mae: 0.1456 - val_loss: 0.0437 - val_mae: 0.1366
Epoch 3/15
82/82 [==============================] - 103s 1s/step - loss: 0.1550 - mae: 0.1441 - val_loss: 0.0486 - val_mae: 0.1692
Epoch 4/15
82/82 [==============================] - 103s 1s/step - loss: 0.1498 - mae: 0.1438 - val_loss: 0.0532 - val_mae: 0.1745
Epoch 5/15
82/82 [==============================] - 103s 1s/step - loss: 0.1469 - mae: 0.1390 - val_loss: 0.0452 - val_mae: 0.1419
Epoch 6/15
82/82 [==============================] - 103s 1s/step - loss: 0.1412 - mae: 0.1345 - val_loss: 0.0439 - val_mae: 0.1412
Epoch 7/15
82/82 [==============================] - 104s 1s/step - loss: 0.1365 - mae: 0.1318 - val_loss: 0.0415 - val_mae: 0.1331
Epoch 8/15
82/82 [==============================] - 103s 1s/step - loss: 0.1406 - m

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 232s 1s/step - loss: 0.1322 - mae: 0.2009 - val_loss: 0.0413 - val_mae: 0.1294
Epoch 2/15
101/101 [==============================] - 128s 1s/step - loss: 0.0924 - mae: 0.1637 - val_loss: 0.0413 - val_mae: 0.1360
Epoch 3/15
101/101 [==============================] - 128s 1s/step - loss: 0.0699 - mae: 0.1458 - val_loss: 0.0437 - val_mae: 0.1469
Epoch 4/15
101/101 [==============================] - 128s 1s/step - loss: 0.0651 - mae: 0.1449 - val_loss: 0.0419 - val_mae: 0.1280
Epoch 5/15
101/101 [==============================] - 128s 1s/step - loss: 0.0558 - mae: 0.1356 - val_loss: 0.0431 - val_mae: 0.1480
Epoch 6/15
101/101 [==============================] - 132s 1s/step - loss: 0.0574 - mae: 0.1351 - val_loss: 0.0399 - val_mae: 0.1221
Epoch 7/15
101/101 [==============================] - 127s 1s/step - loss: 0.0549 - mae: 0.1329 - val_loss: 0.0448 - val_mae: 0.1540
Epoch 8/15
101/101 [==============================] - 127s 1s/step - 

## Bior wavelet, less aggresive denoising

In [25]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'bior3.3', 2, 10)
    X_val= denoise_array(X_val, 0, 'bior3.3', 2, 10)
    X_train = add_mp(X_train, mp_window)
    X_val = add_mp(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_fwd_bior3.3/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_fwd_bior3.3/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_fwd_bior3.3/val_loss.csv')
    np.save("2_Att_layer_MP_fwd_bior3.3/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_fwd_bior3.3/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 241s 1s/step - loss: 0.2824 - mae: 0.2726 - val_loss: 0.0591 - val_mae: 0.1697
Epoch 2/15
101/101 [==============================] - 131s 1s/step - loss: 0.2260 - mae: 0.2142 - val_loss: 0.0562 - val_mae: 0.1661
Epoch 3/15
101/101 [==============================] - 130s 1s/step - loss: 0.2169 - mae: 0.2112 - val_loss: 0.0543 - val_mae: 0.1611
Epoch 4/15
101/101 [==============================] - 129s 1s/step - loss: 0.2159 - mae: 0.2147 - val_loss: 0.0589 - val_mae: 0.1652
Epoch 5/15
101/101 [==============================] - 128s 1s/step - loss: 0.2005 - mae: 0.2024 - val_loss: 0.0659 - val_mae: 0.1761
Epoch 6/15
101/101 [==============================] - 129s 1s/step - loss: 0.1957 - mae: 0.1992 - val_loss: 0.1076 - val_mae: 0.2549
Epoch 7/15
101/101 [==============================] - 131s 1s/step - loss: 0.1918 - mae: 0.1983 - val_loss: 0.0775 - val_mae: 0.1989
Epoch 8/15
101/101 [==============================] - 130s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 254s 1s/step - loss: 0.6626 - mae: 0.4537 - val_loss: 0.4303 - val_mae: 0.4124
Epoch 2/15
107/107 [==============================] - 137s 1s/step - loss: 0.5447 - mae: 0.3838 - val_loss: 0.4378 - val_mae: 0.4078
Epoch 3/15
107/107 [==============================] - 136s 1s/step - loss: 0.5370 - mae: 0.3869 - val_loss: 0.4781 - val_mae: 0.4547
Epoch 4/15
107/107 [==============================] - 136s 1s/step - loss: 0.5050 - mae: 0.3732 - val_loss: 0.5211 - val_mae: 0.5040
Epoch 5/15
107/107 [==============================] - 135s 1s/step - loss: 0.4907 - mae: 0.3719 - val_loss: 0.4499 - val_mae: 0.4045
Epoch 6/15
107/107 [==============================] - 135s 1s/step - loss: 0.4776 - mae: 0.3672 - val_loss: 0.4658 - val_mae: 0.4472
Epoch 7/15
107/107 [==============================] - 137s 1s/step - loss: 0.4689 - mae: 0.3584 - val_loss: 0.4814 - val_mae: 0.4666
Epoch 8/15
107/107 [==============================] - 135s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 244s 1s/step - loss: 0.1744 - mae: 0.2427 - val_loss: 0.3644 - val_mae: 0.2639
Epoch 2/15
101/101 [==============================] - 126s 1s/step - loss: 0.1513 - mae: 0.2126 - val_loss: 0.3707 - val_mae: 0.2527
Epoch 3/15
101/101 [==============================] - 124s 1s/step - loss: 0.1442 - mae: 0.2068 - val_loss: 0.3627 - val_mae: 0.2617
Epoch 4/15
101/101 [==============================] - 126s 1s/step - loss: 0.1383 - mae: 0.2023 - val_loss: 0.3799 - val_mae: 0.2759
Epoch 5/15
101/101 [==============================] - 126s 1s/step - loss: 0.1340 - mae: 0.1984 - val_loss: 0.3870 - val_mae: 0.3157
Epoch 6/15
101/101 [==============================] - 125s 1s/step - loss: 0.1291 - mae: 0.1951 - val_loss: 0.3922 - val_mae: 0.2823
Epoch 7/15
101/101 [==============================] - 125s 1s/step - loss: 0.1239 - mae: 0.1912 - val_loss: 0.3667 - val_mae: 0.2868
Epoch 8/15
101/101 [==============================] - 126s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
82/82 [==============================] - 222s 2s/step - loss: 0.2028 - mae: 0.2018 - val_loss: 0.0683 - val_mae: 0.1718
Epoch 2/15
82/82 [==============================] - 102s 1s/step - loss: 0.1616 - mae: 0.1577 - val_loss: 0.0442 - val_mae: 0.1429
Epoch 3/15
82/82 [==============================] - 102s 1s/step - loss: 0.1518 - mae: 0.1431 - val_loss: 0.0506 - val_mae: 0.1485
Epoch 4/15
82/82 [==============================] - 103s 1s/step - loss: 0.1484 - mae: 0.1406 - val_loss: 0.0580 - val_mae: 0.1783
Epoch 5/15
82/82 [==============================] - 104s 1s/step - loss: 0.1455 - mae: 0.1409 - val_loss: 0.0455 - val_mae: 0.1520
Epoch 6/15
82/82 [==============================] - 103s 1s/step - loss: 0.1413 - mae: 0.1407 - val_loss: 0.0509 - val_mae: 0.1539
Epoch 7/15
82/82 [==============================] - 102s 1s/step - loss: 0.1386 - mae: 0.1378 - val_loss: 0.0464 - val_mae: 0.1443
Epoch 8/15
82/82 [==============================] - 104s 1s/step - loss: 0.1312 - m

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 242s 1s/step - loss: 0.1121 - mae: 0.1895 - val_loss: 0.0554 - val_mae: 0.1776
Epoch 2/15
101/101 [==============================] - 125s 1s/step - loss: 0.0715 - mae: 0.1518 - val_loss: 0.0493 - val_mae: 0.1507
Epoch 3/15
101/101 [==============================] - 126s 1s/step - loss: 0.0680 - mae: 0.1455 - val_loss: 0.0602 - val_mae: 0.1985
Epoch 4/15
101/101 [==============================] - 127s 1s/step - loss: 0.0596 - mae: 0.1396 - val_loss: 0.0529 - val_mae: 0.1776
Epoch 5/15
101/101 [==============================] - 128s 1s/step - loss: 0.0547 - mae: 0.1335 - val_loss: 0.0441 - val_mae: 0.1491
Epoch 6/15
101/101 [==============================] - 129s 1s/step - loss: 0.0582 - mae: 0.1353 - val_loss: 0.0429 - val_mae: 0.1485
Epoch 7/15
101/101 [==============================] - 129s 1s/step - loss: 0.0521 - mae: 0.1314 - val_loss: 0.0754 - val_mae: 0.2365
Epoch 8/15
101/101 [==============================] - 129s 1s/step - 